In [1]:
import os
import sys
os.environ['_JAVA_OPTIONS'] = '-Djava.security.manager=allow -Duser.name=julianromero'
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession.builder.appName("SparkSession").getOrCreate()

from utils.sql_manager import SQLManager
sql_manager = SQLManager(queries_dir='../queries')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
Picked up _JAVA_OPTIONS: -Djava.security.manager=allow -Duser.name=julianromero
Picked up _JAVA_OPTIONS: -Djava.security.manager=allow -Duser.name=julianromero
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/02 20:50:13 WARN Utils: Your hostname, MacBook-Air-de-Julian.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.90 instead (on interface en0)
26/01/02 20:50:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting

In [2]:
#Archivos
hosp_adm = spark.read.csv("../data/nw_hosp/admissions.csv", header=True, inferSchema=True)
icu_stays = spark.read.csv("../data/nw_icu/icustays.csv", header=True, inferSchema=True)
patients = spark.read.csv("../data/nw_hosp/patients.csv", header=True, inferSchema=True)
chart_events = spark.read.csv("../data/nw_icu/chartevents.csv", header=True, inferSchema=True)
lab_events = spark.read.csv("../data/nw_hosp/labevents.csv", header=True, inferSchema=True)
procedure_events = spark.read.csv("../data/nw_icu/procedureevents.csv", header=True, inferSchema=True)

In [3]:
#Vistas temporales
hosp_adm.createOrReplaceTempView("admissions")
icu_stays.createOrReplaceTempView("icu_stays")
patients.createOrReplaceTempView("patients")
chart_events.createOrReplaceTempView("chart_events")
lab_events.createOrReplaceTempView("lab_events")
procedure_events.createOrReplaceTempView("procedure_events")


In [4]:
hosp_adm_data = sql_manager.execute(spark, 'initial_data/admissions.sql')
hosp_adm_data.createOrReplaceTempView("hosp_adm_data")

icu_stays_data = sql_manager.execute(spark, 'initial_data/icu_stays.sql')
icu_stays_data.createOrReplaceTempView("icu_stays_data")

chart_events_data = sql_manager.execute(spark, 'initial_data/chart_events.sql')
chart_events_data.createOrReplaceTempView("chart_events_data")

patients_data = sql_manager.execute(spark, 'initial_data/patients.sql')
patients_data.createOrReplaceTempView("patients_data")

lab_events_data = sql_manager.execute(spark, 'initial_data/lab_events.sql')
lab_events_data.createOrReplaceTempView("lab_events_data")

procedure_events_data = sql_manager.execute(spark, 'initial_data/procedure_events.sql')
procedure_events_data.createOrReplaceTempView("procedure_events_data")


In [5]:
icu_stays.show(5)
stays_count = spark.sql("select subject_id, count(*) as num_stays from icu_stays group by subject_id order by num_stays desc")
stays_count.show(5)
stays_count_df = stays_count.toPandas()
stays_count_df
#23204 pacientes en uci en total

+----------+--------+--------+--------------+-------------+-------------------+-------------------+------------------+
|subject_id| hadm_id| stay_id|first_careunit|last_careunit|             intime|            outtime|               los|
+----------+--------+--------+--------------+-------------+-------------------+-------------------+------------------+
|  30000238|44714428|59801852|         CTICU|        CTICU|2195-10-14 15:12:00|2195-10-16 14:32:00|1.9722222222222223|
|  30000246|40653169|55154064|          CICU|         CICU|2104-05-05 18:23:00|2104-05-07 14:26:00|1.8354166666666667|
|  30000521|46912110|58492193|          SICU|         SICU|2188-04-17 16:46:00|2188-04-18 18:16:00|            1.0625|
|  30000590|44608425|50707570|         NSICU|        NSICU|2177-12-09 12:01:00|2177-12-10 16:43:00|1.1958333333333333|
|  30000828|40615900|57131111|          MICU|          CCU|2124-10-17 23:12:00|2124-10-18 11:57:00|           0.53125|
+----------+--------+--------+--------------+---

,subject_id,num_stays
0,39585304,20
1,35136418,14
2,37150297,12
3,35616012,12
4,35704091,11
...,...,...
23199,39745649,1
23200,39829731,1
23201,39911202,1
23202,36309828,1


In [6]:
admit_date_hospital = sql_manager.execute(spark, 'dates/admision_time.sql')
admit_date_hospital.createOrReplaceTempView("admit_date_hospital")
admit_date_hospital.show(5)
admit_time_ps = admit_date_hospital.toPandas()
admit_time_ps

+----------+------------+-------------------+
|subject_id|admission_id|          admittime|
+----------+------------+-------------------+
|  30000238|    44714428|2195-10-14 07:12:00|
|  30000246|    40653169|2104-05-05 16:18:00|
|  30000246|    43974039|2103-07-08 17:12:00|
|  30000246|    45869139|2103-07-11 10:37:00|
|  30000521|    40796332|2187-08-15 11:33:00|
+----------+------------+-------------------+
only showing top 5 rows


,subject_id,admission_id,admittime
0,30000238,44714428,2195-10-14 07:12:00
1,30000246,40653169,2104-05-05 16:18:00
2,30000246,43974039,2103-07-08 17:12:00
3,30000246,45869139,2103-07-11 10:37:00
4,30000521,40796332,2187-08-15 11:33:00
...,...,...,...
61838,39998775,48786224,2103-02-10 21:07:00
61839,39998775,49181211,2104-02-12 20:43:00
61840,39998775,49555175,2103-10-14 11:50:00
61841,39999363,44506852,2195-02-22 14:39:00


In [7]:
patients_data = sql_manager.execute(spark, 'initial_data/patients_filtered.sql')
patients_data.createOrReplaceTempView("patients_data")
patients_data.show(5)
patients_data_ps = patients_data.toPandas()
patients_data_ps

+----------+------------+-------------------+------+----------+------------------+
|subject_id|admission_id|          admittime|gender|anchor_age|dod_within_30_days|
+----------+------------+-------------------+------+----------+------------------+
|  30000238|    44714428|2195-10-14 07:12:00|     M|        74|              NULL|
|  30000246|    40653169|2104-05-05 16:18:00|     M|        59|              NULL|
|  30000246|    43974039|2103-07-08 17:12:00|     M|        59|              NULL|
|  30000246|    45869139|2103-07-11 10:37:00|     M|        59|              NULL|
|  30000521|    40796332|2187-08-15 11:33:00|     F|        46|              NULL|
+----------+------------+-------------------+------+----------+------------------+
only showing top 5 rows


,subject_id,admission_id,admittime,gender,anchor_age,dod_within_30_days
0,30000238,44714428,2195-10-14 07:12:00,M,74,None
1,30000246,40653169,2104-05-05 16:18:00,M,59,None
2,30000246,43974039,2103-07-08 17:12:00,M,59,None
3,30000246,45869139,2103-07-11 10:37:00,M,59,None
4,30000521,40796332,2187-08-15 11:33:00,F,46,None
...,...,...,...,...,...,...
61838,39998775,48786224,2103-02-10 21:07:00,M,75,None
61839,39998775,49181211,2104-02-12 20:43:00,M,75,None
61840,39998775,49555175,2103-10-14 11:50:00,M,75,None
61841,39999363,44506852,2195-02-22 14:39:00,F,73,None


In [8]:
chart_events_post_admission = sql_manager.execute(spark, 'joins/admission_chartevents.sql')
chart_events_post_admission.createOrReplaceTempView("chart_events_adm")
chart_events_post_admission.show(5)

+----------+------------+--------+-------------------+----------------+--------------------+--------------------+
|subject_id|admission_id| stay_id|          admittime|num_chart_events|       chart_itemids|        chart_values|
+----------+------------+--------+-------------------+----------------+--------------------+--------------------+
|  30000521|    46912110|58492193|2188-04-17 16:46:00|              53|[320045, 320179, ...|[92.0, 160.0, 79....|
|  30000828|    40615900|57131111|2124-10-17 23:12:00|              17|[320045, 320179, ...|[97.0, 131.0, 87....|
|  30003460|    43300755|52287466|2139-04-28 18:30:00|              47|[320045, 320050, ...|[66.0, 126.0, 58....|
|  30004640|    41454271|57661199|2128-06-04 14:14:00|             162|[320045, 320179, ...|[63.0, 127.0, 78....|
|  30006179|    40825743|57485621|2133-05-08 06:47:00|              85|[320045, 320179, ...|[79.0, 145.0, 66....|
+----------+------------+--------+-------------------+----------------+-----------------

In [9]:
lab_events_adm = sql_manager.execute(spark, 'joins/admission_labevents.sql')
lab_events_adm.createOrReplaceTempView("lab_events_adm")
lab_events_adm.show(5)
lab_events_adm_df = lab_events_adm.toPandas()
lab_events_adm_df

+----------+------------+--------+-------------------+--------------+--------------------+--------------------+
|subject_id|admission_id| stay_id|          admittime|num_lab_events|         lab_itemids|          lab_values|
+----------+------------+--------+-------------------+--------------+--------------------+--------------------+
|  30000521|    46912110|58492193|2188-04-17 16:46:00|            20|[100040, 100003, ...|[9.0, 8.0, 94.6, ...|
|  30000828|    40615900|57131111|2124-10-17 23:12:00|             0|                  []|                  []|
|  30003460|    43300755|52287466|2139-04-28 18:30:00|            22|[100046, 100003, ...|[30.7, 8.8, 1.6, ...|
|  30004640|    41454271|57661199|2128-06-04 14:14:00|            16|[100001, 100013, ...|[158.0, 223.0, 8....|
|  30006179|    40825743|57485621|2133-05-08 06:47:00|            31|[100006, 100005, ...|[23.3, 35.0, 23.1...|
+----------+------------+--------+-------------------+--------------+--------------------+--------------

,subject_id,admission_id,stay_id,admittime,num_lab_events,lab_itemids,lab_values
0,30000521,46912110,58492193,2188-04-17 16:46:00,20,"[100040, 100003, 100017, 100014, 100004, 10001...","[9.0, 8.0, 94.6, 160.0, 9.0, 23.0, 11.9, 170.0..."
1,30000828,40615900,57131111,2124-10-17 23:12:00,0,[],[]
2,30003460,43300755,52287466,2139-04-28 18:30:00,22,"[100046, 100003, 100009, 100012, 100017, 10004...","[30.7, 8.8, 1.6, 101.0, 103.0, 10.0, 34.5, 155..."
3,30004640,41454271,57661199,2128-06-04 14:14:00,16,"[100001, 100013, 100004, 100040, 100001, 10000...","[158.0, 223.0, 8.8, 1.8, 100.0, 8.0, 133.0, 23..."
4,30006179,40825743,57485621,2133-05-08 06:47:00,31,"[100006, 100005, 100034, 100018, 100007, 10001...","[23.3, 35.0, 23.1, 32.6, 7.6, 75.0, 107.0, 2.1..."
...,...,...,...,...,...,...,...
28607,39919947,44443782,51400661,2186-01-09 15:14:00,21,"[100018, 100010, 100006, 100016, 100014, 10000...","[33.9, 143.0, 38.0, 7.7, 167.0, 13.2, 12.9, 10..."
28608,39926572,45102982,56018746,2195-11-21 15:32:00,21,"[100001, 100003, 100040, 100010, 100001, 10000...","[92.0, 8.4, 8.0, 143.0, 128.0, 8.0, 0.5, 3.5, ..."
28609,39950368,45844966,57619647,2195-04-05 03:00:00,24,"[100019, 100003, 100017, 100006, 100007, 10004...","[0.0, 7.9, 87.4, 36.8, 11.7, 13.0, 4.21, 3.3, ..."
28610,39955880,43010884,53075985,2199-06-18 23:16:00,33,"[100031, 100021, 100012, 100040, 100042, 10002...","[1.1, 3.4, 94.0, 13.0, 165.0, 1.4, 4.8, 25.0, ..."


In [10]:
procedure_events_adm = sql_manager.execute(spark, 'joins/admission_procedure_events.sql')
procedure_events_adm.createOrReplaceTempView("procedure_events_adm")
procedure_events_adm.show(5)

+----------+------------+--------+-------------------+---------------+--------------------+--------------------+
|subject_id|admission_id| stay_id|          admittime|num_prod_events| prod_events_itemids|         prod_values|
+----------+------------+--------+-------------------+---------------+--------------------+--------------------+
|  30000238|    44714428|59801852|2195-10-14 15:12:00|             64|[772725, 707814, ...|[576.0, 500.0, 17...|
|  30000246|    40653169|55154064|2104-05-05 18:23:00|              0|                  []|                  []|
|  30001076|    41565504|56667136|2104-10-20 22:15:00|              0|                  []|                  []|
|  30001076|    41565504|59550340|2104-10-24 18:09:00|              0|                  []|                  []|
|  30001292|    47098230|57234823|2116-02-03 04:23:00|             33|[766246, 766246, ...|[408.0, 330.0, 16...|
+----------+------------+--------+-------------------+---------------+--------------------+-----

In [11]:
dataset = sql_manager.execute(spark, 'final/dataset.sql')
dataset.show(5)
""" dataset_df = dataset.toPandas()
dataset_df.to_csv("data/final_dataset.csv", index=False) """


+----------+------------+-------------------+--------+-------------------+---------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+------+----------+------------------+
|subject_id|admission_id|          admittime| stay_id|     length_of_stay|num_prod_events| prod_events_itemids|         prod_values|num_chart_events|       chart_itemids|        chart_values|num_lab_events|         lab_itemids|          lab_values|gender|anchor_age|dod_within_30_days|
+----------+------------+-------------------+--------+-------------------+---------------+--------------------+--------------------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+------+----------+------------------+
|  30005809|    44231909|2143-03-25 13:06:00|52987879| 1.0659722222222223|             38|[763150, 763189, ...|[440.0, 431.0, 19...|          

' dataset_df = dataset.toPandas()\ndataset_df.to_csv("data/final_dataset.csv", index=False) '

In [15]:
from pyspark.sql.functions import col, coalesce, when, count as spark_count, row_number
from pyspark.sql.window import Window
import pandas as pd

# Contar valores no-nulos por fila
dataset_with_nulls = dataset.withColumn(
    "non_null_count",
    sum(when(col(c).isNotNull(), 1).otherwise(0) for c in dataset.columns)
)

# Definir window function: ordenar por stay_id, con mayor número de non-nulls primero
window_spec = Window.partitionBy("stay_id").orderBy(col("non_null_count").desc())

# Agregar row number
dataset_dedup = dataset_with_nulls.withColumn("rn", row_number().over(window_spec))

# Quedarse solo con la primera (la más completa)
dataset_dedup_final = dataset_dedup.filter(col("rn") == 1).drop("rn", "non_null_count")

# Actualizar la vista temporal
dataset_dedup_final.createOrReplaceTempView("dataset")

dataset_df = dataset_dedup_final.toPandas()
#dataset_df.to_csv("data/final_dataset_dedup.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data'

In [16]:
dataset_df

,subject_id,admission_id,admittime,stay_id,length_of_stay,num_prod_events,prod_events_itemids,prod_values,num_chart_events,chart_itemids,chart_values,num_lab_events,lab_itemids,lab_values,gender,anchor_age,dod_within_30_days
0,36375942,44967998,2107-06-06 15:37:00,50000094,1.109028,20,"[755659, 720910, 755659, 720910, 785854, 77799...","[478.0, 346.0, 118.0, 115.0, 84.0, 45.0, 23.0]",0,[],[],82,"[100029, 100047, 100031, 100045, 100044, 10005...","[21.0, 3.0, 0.7, 130.0, 1.13, 131.0, 16.0, 23....",M,66,None
1,39110537,48405353,2184-05-08 12:47:00,50006700,0.209722,22,"[799878, 788910, 784206, 799878, 760028, 70781...","[516.0, 396.0, 110.0, 102.0, 78.0, 59.0]",34,"[320045, 320050, 320051, 320277, 320045, 32005...","[97.0, 87.0, 47.0, 71.0, 97.0, 114.0, 62.0, 66...",20,"[100010, 100032, 100003, 100034, 100012, 10001...","[140.0, 586.0, 7.1, 14.4, 99.0, 29.0, 0.98, 4....",F,77,None
2,33922515,47797359,2183-09-03 19:06:00,50007064,0.105556,7,"[752616, 785854, 781236, 777999, 774054, 75490...",[],146,"[320045, 320179, 320180, 320210, 320277, 32004...","[121.0, 65.0, 32.0, 24.0, 99.0, 118.0, 74.0, 3...",34,"[100009, 100075, 100012, 100032, 100035, 10007...","[3.5, 16265.0, 96.0, 4546.0, 0.08, 2.8, 34.0, ...",M,78,None
3,31228238,42646040,2122-04-02 17:04:00,50010151,2.860417,0,[],[],51,"[320045, 320050, 320051, 320179, 320180, 32021...","[93.0, 122.0, 53.0, 110.0, 58.0, 11.0, 100.0, ...",23,"[100009, 100001, 100011, 100003, 100002, 10001...","[2.0, 150.0, 4.0, 8.1, 0.84, 110.0, 3.8, 142.0...",F,74,None
4,39818094,47119780,2191-09-17 13:22:00,50014044,1.905556,0,[],[],63,"[320045, 320179, 320180, 320210, 320277, 32004...","[72.0, 154.0, 103.0, 16.0, 95.0, 73.0, 133.0, ...",1,[100053],[9999999.0],M,33,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28607,30283833,40512571,2115-08-31 19:01:00,59980643,2.950694,0,[],[],31,"[320045, 320210, 320277, 323761, 320045, 32017...","[112.0, 29.0, 96.0, 98.0, 109.0, 110.0, 66.0, ...",0,[],[],F,78,None
28608,34068385,48743757,2129-08-26 05:52:00,59982494,1.413889,11,"[763150, 777999, 766246, 766246, 766246, 76624...","[451.0, 306.0, 140.0, 131.0, 80.0, 54.0, 51.0,...",102,"[323761, 326531, 320045, 320179, 320180, 32027...","[99.4, 2220.8, 153.0, 129.0, 63.0, 100.0, 127....",106,"[100074, 100061, 100015, 100013, 100013, 10000...","[9999999.0, 1.015, 1.0, 14.0, 15.0, 8.0, 14.0,...",F,19,None
28609,39072991,45908959,2195-09-05 00:04:00,59996555,0.584028,0,[],[],27,"[320045, 320179, 320180, 320277, 320210, 32376...","[78.0, 138.0, 62.0, 93.0, 18.0, 98.4, 69.0, 96...",19,"[100036, 100023, 100035, 100038, 100026, 10000...","[0.03, 27.0, 0.03, 3.81, 7.0, 29.5, 0.42, 32.8...",M,86,None
28610,39883726,46456736,2128-02-04 03:17:00,59997276,5.613194,31,"[770354, 785510, 775677, 739799, 766246, 76624...","[455.0, 370.0, 172.0, 94.0, 91.0, 56.0, 41.0, ...",0,[],[],37,"[100004, 100040, 100026, 100003, 100012, 10003...","[13.0, 5.0, 4.2, 7.9, 107.0, 0.1, 37.6, 26.0, ...",M,65,None


In [24]:
lab_events = sql_manager.execute(spark, 'joins/icu_stays_labevents.sql')
lab_events.createOrReplaceTempView("lab_events_icu")
count_item_id = spark.sql("select itemid, count(*) as count from lab_events_icu group by itemid order by count desc")
lab_events_df = count_item_id.toPandas()
lab_events_df

,itemid,count
0,100001,41770
1,100011,21164
2,100010,20715
3,100003,19886
4,100002,19861
...,...,...
201,100306,1
202,100172,1
203,100338,1
204,100334,1


In [26]:
chart_events = sql_manager.execute(spark, 'joins/icu_stays_chartevents.sql')
chart_events.createOrReplaceTempView("chart_events_icu")
count_item_id_chart = spark.sql("select itemid, count(*) as count from chart_events_icu group by itemid order by count desc")
chart_events_df = count_item_id_chart.toPandas()
chart_events_df

,itemid,count
0,320045,311904
1,320277,300912
2,320210,269071
3,320179,233411
4,320180,233411
5,323761,102953
6,320050,98946
7,320051,98817
8,326531,10389
9,300001,8766
